In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from keras_tuner import RandomSearch

In [2]:
Trails=3 # Number of models to train
epochs=2 # number of epoch per model

# Get the MNIST dataset

In [3]:
(xtrain,ytrain),(xval,yval)=mnist.load_data()
xtrain=np.expand_dims(xtrain.astype('float32')/255.,-1)
xval=np.expand_dims(xval.astype('float32')/255.,-1)
ytrain=to_categorical(ytrain,10)
yval=to_categorical(yval,10)


In [4]:
def build_model(hp):
    """
    Function to build a TF model based on hyperparameter values
    
    Args:
        hp (hyperparameter): values
        
    Returns:
        Model: compiled model
    """
    num_layers = hp.Int('num_layers', 2, 8, default=6)
    lr = hp.Choice('learning_rate', [0.0001, 0.05])
    inputs = layers.Input(shape=(28, 28, 1))
    x = inputs
    
    for idx in range(num_layers):
        idx = str(idx)
        filters = hp.Int('filters_'+idx, 32, 256, step=32, default=64)
        x = layers.Conv2D(filters=filters, kernel_size=3, padding='same', activation='relu')(x)

        if x.shape[1] >= 8:
            pool_type = hp.Choice('pool_'+idx, values=['max', 'avg'])
            if pool_type == 'max':
                x = layers.MaxPooling2D(2)(x)
            elif pool_type == 'avg':
                x = layers.AveragePooling2D(2)(x)

    x = layers.Flatten()(x)
    outputs = layers.Dense(10, activation='softmax')(x)

    # Build model
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=Adam(lr), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [5]:
tuner=RandomSearch(build_model,objective='val_accuracy',max_trials=Trails,
 project_name='MNIST_results')

Reloading Tuner from .\MNIST_results\tuner0.json


In [6]:
tuner.search_space_summary()

Search space summary
Default search space size: 10
num_layers (Int)
{'default': 6, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.0001, 'conditions': [], 'values': [0.0001, 0.05], 'ordered': True}
filters_0 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
pool_0 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
filters_1 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
pool_1 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
filters_2 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
filters_3 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
filters_4 (Int)
{'default': 64, 'conditions': [], 'min_value

In [7]:
tuner.search(xtrain,ytrain,batch_size=128,epochs=2,validation_data=(xval,yval))

Trial 3 Complete [00h 08m 24s]
val_accuracy: 0.9790999889373779

Best val_accuracy So Far: 0.9790999889373779
Total elapsed time: 1d 23h 41m 11s


In [8]:
tuner.results_summary()

Results summary
Results in .\MNIST_results
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 2
learning_rate: 0.05
filters_0: 192
pool_0: avg
filters_1: 224
pool_1: avg
filters_2: 128
filters_3: 128
filters_4: 192
filters_5: 96
Score: 0.9790999889373779

Trial 0 summary
Hyperparameters:
num_layers: 6
learning_rate: 0.0001
filters_0: 64
pool_0: max
filters_1: 192
pool_1: avg
filters_2: 256
filters_3: 128
filters_4: 128
filters_5: 32
Score: 0.9732999801635742

Trial 1 summary
Hyperparameters:
num_layers: 2
learning_rate: 0.0001
filters_0: 256
pool_0: max
filters_1: 192
pool_1: max
filters_2: 32
filters_3: 256
filters_4: 160
filters_5: 64
Score: 0.9721999764442444


In [9]:
best_model=tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 192)       1920      
                                                                 
 average_pooling2d (Average  (None, 14, 14, 192)       0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 224)       387296    
                                                                 
 average_pooling2d_1 (Avera  (None, 7, 7, 224)         0         
 gePooling2D)                                                    
                                                                 
 flatten (Flatten)           (None, 10976)             0     

# 1.Build a Neural network with Hyper-parameter fine tuning model

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_breast_cancer
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from kerastuner.tuners import RandomSearch

C:\Users\SIRISHA\AppData\Local\Temp\ipykernel_31992\3231417136.py:8: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [11]:
# Load the Breast Cancer dataset
cancer = load_breast_cancer()
data = pd.DataFrame(data=np.c_[cancer['data'], cancer['target']], columns=cancer['feature_names'].tolist() + ['target'])


In [12]:
# Preprocess the data
X = data.drop('target', axis=1)
y = LabelEncoder().fit_transform(data['target'])
y = to_categorical(y)

In [13]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Define the model building function
def build_model(hp):
    model = keras.Sequential()
    
    # Tune the number of units in the first Dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu', input_dim=X_train.shape[1]))
    
    # Tune the number of layers
    hp_layers = hp.Int('num_layers', min_value=1, max_value=3)
    for _ in range(hp_layers):
        model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    
    model.add(keras.layers.Dense(2, activation='softmax'))  # 2 classes for Breast Cancer dataset

    # Tune the learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [23]:
# Initialize the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='hyperparameter_tuning',
    project_name='cancer_tuning'
)

Reloading Tuner from hyperparameter_tuning\cancer_tuning\tuner0.json


In [24]:
# Perform hyperparameter tuning
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [26]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [27]:
tuner.search_space_summary

<bound method BaseTuner.search_space_summary of <keras_tuner.src.tuners.randomsearch.RandomSearch object at 0x000002472915FBD0>>

In [29]:
# Get the best model
#best_model = tuner.get_best_models(num_models=1)[0]
best_model=tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               7936      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 2)                 514       
                                                                 
Total params: 74242 (290.01 KB)
Trainable params: 74242 (290.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
!pip install torch torchvision torchaudio

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ---------------------------------------- 0.0/192.3 MB ? eta -:--:--
     ---------------------------------------- 0.1/192.3 MB 1.6 MB/s eta 0:02:01
     ---------------------------------------- 0.1/192.3 MB 1.6 MB/s eta 0:02:01
     ---------------------------------------- 0.1/192.3 MB 1.6 MB/s eta 0:02:01
     ---------------------------------------- 0.1/192.3 MB 1.6 MB/s eta 0:02:01
     ---------------------------------------- 0.1/192.3 MB 1.6 MB/s eta 0:02:01
     -------------------------------------- 0.1/192.3 MB 217.9 kB/s eta 0:14:43
     -------------------------------------- 0.1/192.3 MB 343.4 kB/s eta 0:09:20
     -------------------------------------- 0.2/192.3 MB 455.1 kB/s eta 0:07:03
     -------------------------------------- 0.2/192.3 MB 471.4 kB/s eta 0:06:48
     -------------------------------------- 0.2/192.3 MB 471.4 kB/s eta 0:06:48
     -------------------------------------- 0.2/192.3 MB 471.4 kB/s 

     ------------- ------------------------- 65.9/192.3 MB 1.0 MB/s eta 0:02:02
     ------------- ------------------------- 65.9/192.3 MB 1.0 MB/s eta 0:02:03
     ------------- ------------------------- 65.9/192.3 MB 1.0 MB/s eta 0:02:03
     ------------- ------------------------- 65.9/192.3 MB 1.0 MB/s eta 0:02:03
     ------------- ------------------------- 66.0/192.3 MB 1.0 MB/s eta 0:02:04
     ------------- ------------------------- 66.0/192.3 MB 1.0 MB/s eta 0:02:02
     ------------- ------------------------- 66.1/192.3 MB 1.0 MB/s eta 0:02:02
     ------------- ------------------------- 66.2/192.3 MB 1.0 MB/s eta 0:02:02
     ------------- ------------------------- 66.3/192.3 MB 1.0 MB/s eta 0:02:02
     ------------- ------------------------- 66.3/192.3 MB 1.0 MB/s eta 0:02:02
     ------------- ------------------------- 66.3/192.3 MB 1.0 MB/s eta 0:02:02
     ------------- ------------------------- 66.3/192.3 MB 1.0 MB/s eta 0:02:02
     ------------- ---------------------

# 2.Build an image classifier model with Pytorch

In [32]:
import ssl
import urllib.request as urllib

# Disable SSL verification
ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
file_name = 'cifar-10-python.tar.gz'

urllib.urlretrieve(url, file_name)


('cifar-10-python.tar.gz', <http.client.HTTPMessage at 0x247244cc410>)

# with CIFAR-10 dataset

In [44]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [46]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define layers
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()


In [47]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [49]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Train the network
for epoch in range(2):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = net(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize

        running_loss += loss.item()
        if i % 2000 == 1999:  # Print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 2000}')
            running_loss = 0.0

print('Finished Training')


The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
[1, 2000] loss: 2.240172295033932
[1, 4000] loss: 1.9191793392598628
[1, 6000] loss: 1.7106768944263457
[1, 8000] loss: 1.6246396196335555
[1, 10000] loss: 1.5345035525411368
[1, 12000] loss: 1.484433392137289
[2, 2000] loss: 1.395871634438634
[2, 4000] loss: 1.3786333858519793
[2, 6000] loss: 1.3851274054199456
[2, 8000] loss: 1.3511680152118206
[2, 10000] loss: 1.3116444626376034
[2, 12000] loss: 1.3014257483929397
Finished Training


In [50]:
# Test the network on test data
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {(100 * correct / total)}%')


Accuracy of the network on the 10000 test images: 54.21%


# with FashionMNIST dataset

In [38]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# Define transformations for data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize images
])

# Download and load FashionMNIST dataset
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                             download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                            download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress',
           'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')

# Define a simple CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the network
net = Net()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Train the network
for epoch in range(2):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = net(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize

        running_loss += loss.item()
        if i % 2000 == 1999:  # Print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 2000}')
            running_loss = 0.0

print('Finished Training')

# Test the network on test data
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {(100 * correct / total)}%')


100%|█████████████████████████████████████████████████████████████████| 26421880/26421880 [00:03<00:00, 7942109.83it/s]


Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 184732.64it/s]


Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



100%|███████████████████████████████████████████████████████████████████| 4422102/4422102 [00:01<00:00, 3014188.00it/s]


Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<00:00, 4749730.97it/s]


Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw

[1, 2000] loss: 1.400774951535277
[1, 4000] loss: 0.6787337659470504
[1, 6000] loss: 0.5651286336596822
[1, 8000] loss: 0.5222390857039136
[1, 10000] loss: 0.4830375804807991
[1, 12000] loss: 0.45044085882941726
[1, 14000] loss: 0.43027133203280393
[2, 2000] loss: 0.38150036406826987
[2, 4000] loss: 0.3997376966188603
[2, 6000] loss: 0.3711103633627208
[2, 8000] loss: 0.38129062078855347
[2, 10000] loss: 0.35170009461767404
[2, 12000] loss: 0.3615673464344218
[2, 14000] loss: 0.3480155385346843
Finished Training
Accuracy of the network on the 10000 test images: 85.99%
